# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](../images/tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [40]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras import models
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [41]:
df = pd.read_csv('tic_tac_toe.csv')
df.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB


In [43]:
col = list(df.columns)
col.remove('class')
for i in col:
    df.replace({i:{'b': 0, 'x':1, 'o':2}}, inplace=True)

In [32]:
#for i in df['class']:
    #df.replace({i:{True:1, False:0}}, inplace=True)
    #doesn't work

In [44]:
class_int = preprocessing.LabelBinarizer()
df['class'] = class_int.fit_transform(df['class'])

In [45]:
df.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,1,1,1,1,2,2,1,2,2,1
1,1,1,1,1,2,2,2,1,2,1
2,1,1,1,1,2,2,2,2,1,1
3,1,1,1,1,2,2,2,0,0,1
4,1,1,1,1,2,2,0,2,0,1


In [46]:
# x = df.values 
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# df = pd.DataFrame(data=x_scaled, columns=df.columns)

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   TL      958 non-null    int64
 1   TM      958 non-null    int64
 2   TR      958 non-null    int64
 3   ML      958 non-null    int64
 4   MM      958 non-null    int64
 5   MR      958 non-null    int64
 6   BL      958 non-null    int64
 7   BM      958 non-null    int64
 8   BR      958 non-null    int64
 9   class   958 non-null    int32
dtypes: int32(1), int64(9)
memory usage: 71.2 KB


In [48]:
features = df.drop(['class'], axis=1).copy()
target = pd.DataFrame(df['class'])

In [49]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.8, random_state=111)

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [76]:
#2.3.
df_model = models.Sequential()
df_model.add(Dense(units=111, kernel_initializer='uniform', activation='relu', input_dim=9))
df_model.add(Dense(units=111, kernel_initializer='uniform', activation='relu'))
df_model.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
#4.
df_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#5.
df_model.fit(X_train ,y_train, batch_size=2, epochs=50)

loss, accuracy = df_model.evaluate(X_test, y_test, verbose=0)

print(loss)
print(accuracy)

df_model.save('tic-tac-toe.model')

Epoch 1/50
191/191 [==============================] - 0s 1ms/step - loss: 0.6707 - accuracy: 0.6545
Epoch 2/50
191/191 [==============================] - 0s 658us/step - loss: 0.6470 - accuracy: 0.6545
Epoch 3/50
191/191 [==============================] - 0s 674us/step - loss: 0.6237 - accuracy: 0.6545
Epoch 4/50
191/191 [==============================] - 0s 605us/step - loss: 0.6098 - accuracy: 0.6545
Epoch 5/50
191/191 [==============================] - 0s 553us/step - loss: 0.5832 - accuracy: 0.6754
Epoch 6/50
191/191 [==============================] - 0s 615us/step - loss: 0.5509 - accuracy: 0.7277
Epoch 7/50
191/191 [==============================] - 0s 591us/step - loss: 0.5269 - accuracy: 0.7382
Epoch 8/50
191/191 [==============================] - 0s 517us/step - loss: 0.5026 - accuracy: 0.7382
Epoch 9/50
191/191 [==============================] - 0s 590us/step - loss: 0.4722 - accuracy: 0.7853
Epoch 10/50
191/191 [==============================] - 0s 578us/step - loss: 0.4377 

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [77]:
my_model = models.load_model('tic-tac-toe.model')

In [78]:
y_pred = my_model.predict(X_test[111:151])
for i,j in zip(y_test[111:151].values, y_pred):
    print(f'actual {i} vs predicted {j}')

actual [0] vs predicted [0.09351388]
actual [0] vs predicted [0.99999857]
actual [1] vs predicted [0.9999881]
actual [1] vs predicted [1.]
actual [1] vs predicted [0.9991193]
actual [1] vs predicted [0.9825574]
actual [0] vs predicted [0.13643602]
actual [1] vs predicted [0.9978883]
actual [1] vs predicted [0.67862815]
actual [1] vs predicted [0.99387604]
actual [1] vs predicted [0.99986076]
actual [1] vs predicted [0.99994266]
actual [0] vs predicted [0.23780218]
actual [1] vs predicted [0.93378866]
actual [1] vs predicted [0.9992094]
actual [0] vs predicted [0.9999201]
actual [1] vs predicted [0.99801844]
actual [1] vs predicted [1.]
actual [1] vs predicted [0.8790058]
actual [0] vs predicted [0.9992168]
actual [1] vs predicted [1.]
actual [1] vs predicted [0.9999924]
actual [1] vs predicted [1.]
actual [0] vs predicted [0.00637788]
actual [1] vs predicted [1.]
actual [1] vs predicted [0.80870783]
actual [1] vs predicted [0.9999459]
actual [1] vs predicted [0.9999881]
actual [1] vs p

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

I played a little bit with the parameters (epochs, batch_sizeand units), but at the end, the best improvement came from using sigmoid as activation for the output. Now that the model has achieved low loss (<0.1) and high accuracy (>0.95). I'm going to leave it here, I'll give it a look once I finish the final project.

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here